# Export onnx with your trained weights

In [2]:
# revise the code according to your model
# the second line is the location of exporting script, base on your nature of your task (classification/ detection/ segmentation)
# the third line is the location of configs of your model
# the fourth line is the location of weights (pth) of your model
# the fifth line is a sample of image
# the sixth line is exporting location
# the seventh line is choosing model to run on cpu or cuda

In [1]:
%run mmdeploy/tools/deploy.py \
mmdeploy/configs/mmpretrain/classification_onnxruntime_dynamic.py \
mmpretrain/work_dirs/resnet101_8xb32_planenbike/resnet101_8xb32_planenbike.py \
mmpretrain/work_dirs/resnet101_8xb32_planenbike/epoch_200.pth \
mmpretrain/caltech101/airplanes/image_0800.jpg \
--work-dir mmdeploy_model/resnet \
--device cuda

06/02 16:11:15 - mmengine - INFO - Start pipeline mmdeploy.apis.pytorch2onnx.torch2onnx in subprocess
06/02 16:11:16 - mmengine - WARNING - Failed to search registry with scope "mmpretrain" in the "Codebases" registry tree. As a workaround, the current "Codebases" registry in "mmdeploy" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmpretrain" is a correct scope, or whether the registry is initialized.
06/02 16:11:16 - mmengine - WARNING - Failed to search registry with scope "mmpretrain" in the "mmpretrain_tasks" registry tree. As a workaround, the current "mmpretrain_tasks" registry in "mmdeploy" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmpretrain" is a correct scope, or whether the registry is initialized.
Loads checkpoint by local backend from path: mmpretrain/work_dirs/resnet101_8xb32_planenbike/epoch_200.pth
06/02 16:11:16 - mmengine - WAR

⚠️ **Note on Running Deployment Cells**

Due to how Python's `multiprocessing` module interacts with interactive environments like Jupyter Notebook, running multiple `%run mmdeploy/tools/deploy.py` commands consecutively **without restarting the kernel** may lead to the following error:

" AttributeError: module 'main' has no attribute 'spec' "


This is a known limitation when using `spawn` multiprocessing context in scripts executed via `%run`.

🛠 **Recommended Action**:  
After executing a deployment script, please **restart the Jupyter kernel** before running another `%run mmdeploy/tools/deploy.py ...` command. You can do this by clicking:

> Kernel → Restart Kernel

This will prevent the multiprocessing error and ensure stable behavior.


In [1]:
%run mmdeploy/tools/deploy.py \
mmdeploy/configs/mmpretrain/classification_onnxruntime_dynamic.py \
mmpretrain/work_dirs/resnet101-csra_1xb16_voc07-448px/resnet101-csra_1xb16_voc07-448px.py \
mmpretrain/work_dirs/resnet101-csra_1xb16_voc07-448px/epoch_16.pth \
mmpretrain/data/VOC2007/JPEGImages/000001.jpg \
--work-dir mmdeploy_model/csra \
--device cuda

06/02 16:06:12 - mmengine - INFO - Start pipeline mmdeploy.apis.pytorch2onnx.torch2onnx in subprocess
06/02 16:06:12 - mmengine - WARNING - Failed to search registry with scope "mmpretrain" in the "Codebases" registry tree. As a workaround, the current "Codebases" registry in "mmdeploy" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmpretrain" is a correct scope, or whether the registry is initialized.
06/02 16:06:12 - mmengine - WARNING - Failed to search registry with scope "mmpretrain" in the "mmpretrain_tasks" registry tree. As a workaround, the current "mmpretrain_tasks" registry in "mmdeploy" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmpretrain" is a correct scope, or whether the registry is initialized.
Loads checkpoint by local backend from path: mmpretrain/work_dirs/resnet101-csra_1xb16_voc07-448px/epoch_16.pth
06/02 16:06:13 - mmengine 

2025-06-02 16:06:17.434537556 [W:onnxruntime:, session_state.cc:1280 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2025-06-02 16:06:17.434555703 [W:onnxruntime:, session_state.cc:1282 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.


06/02 16:06:17 - mmengine - WARNING - expect input device cuda but get cuda:0.
06/02 16:06:19 - mmengine - INFO - visualize onnxruntime model success.
06/02 16:06:19 - mmengine - INFO - visualize pytorch model start.
06/02 16:06:20 - mmengine - WARNING - Failed to search registry with scope "mmpretrain" in the "Codebases" registry tree. As a workaround, the current "Codebases" registry in "mmdeploy" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmpretrain" is a correct scope, or whether the registry is initialized.
06/02 16:06:20 - mmengine - WARNING - Failed to search registry with scope "mmpretrain" in the "mmpretrain_tasks" registry tree. As a workaround, the current "mmpretrain_tasks" registry in "mmdeploy" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmpretrain" is a correct scope, or whether the registry is initialized.
Loads checkpoint by loc

# Inference the onnx

In [2]:
# providers set provider priority cuda or cpu
import onnxruntime
sess = onnxruntime.InferenceSession("mmdeploy_model/resnet/end2end.onnx", providers=["CUDAExecutionProvider"])
print(sess.get_providers())

['CUDAExecutionProvider', 'CPUExecutionProvider']


In [3]:
'''
from mmdeploy.apis import inference_model
is the official method to run an onnx, but it require torch
'''
#Classification
from mmdeploy.apis import inference_model
result = inference_model(
    model_cfg='mmpretrain/work_dirs/resnet101_8xb32_planenbike/resnet101_8xb32_planenbike.py',
    deploy_cfg='mmdeploy/configs/mmpretrain/classification_onnxruntime_dynamic.py',
    backend_files=['mmdeploy_model/resnet/end2end.onnx'],
    img='mmpretrain/caltech101/airplanes/image_0800.jpg',
    device='cuda')
print(result)

06/02 16:07:39 - mmengine - WARNING - Failed to search registry with scope "mmpretrain" in the "Codebases" registry tree. As a workaround, the current "Codebases" registry in "mmdeploy" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmpretrain" is a correct scope, or whether the registry is initialized.
06/02 16:07:39 - mmengine - WARNING - Failed to search registry with scope "mmpretrain" in the "mmpretrain_tasks" registry tree. As a workaround, the current "mmpretrain_tasks" registry in "mmdeploy" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmpretrain" is a correct scope, or whether the registry is initialized.
06/02 16:07:39 - mmengine - WARNING - Failed to search registry with scope "mmpretrain" in the "backend_classifiers" registry tree. As a workaround, the current "backend_classifiers" registry in "mmdeploy" is used to build instance. This ma

In [4]:
'''
this is an unofficial method to run an onnx that require no torch
however, you need to check the 'model_cfg' to understand how the image was preprocessed
which include normalization method and resize and padding method
you may search keywords "data_preprocessor" and "mean" and "std" for normalization 
and "pipeline" and "scale" for resize and padding method
'''
#Classification 
import onnxruntime
import numpy as np
from PIL import Image

# we resize with longer edge == target size, keep ratio, and pad on short edge
def resize_and_pad(image, target_size, pad_color=(114, 114, 114)):
    # Get original size
    w, h = image.size

    # Determine scale factor and new size
    scale = target_size / max(w, h)
    new_w, new_h = int(w * scale), int(h * scale)

    # Resize the image with aspect ratio preserved
    image = image.resize((new_w, new_h), Image.BILINEAR)

    # Create a new square image and paste the resized image in the center
    new_image = Image.new("RGB", (target_size, target_size), pad_color)
    paste_x = (target_size - new_w) // 2
    paste_y = (target_size - new_h) // 2
    new_image.paste(image, (paste_x, paste_y))

    return new_image
    
# Load the ONNX model with CUDA execution provider
sess = onnxruntime.InferenceSession("mmdeploy_model/resnet/end2end.onnx", providers=["CPUExecutionProvider"])
print("Available providers:", sess.get_providers())

# Define the image path
image_path = "mmpretrain/caltech101/airplanes/image_0799.jpg"

# Load and preprocess the image
image = Image.open(image_path)
image = resize_and_pad(image, target_size=224)

# Convert the input tensor to a numpy array (ONNXRuntime uses numpy arrays)
input_array = np.transpose(image, (2,0,1))
input_array = np.expand_dims(input_array,axis=0)

'''
according to config
    data_preprocessor = dict(
        mean=[
            125.307,
            122.961,
            113.8575,
        ],
        std=[
            51.5865,
            50.847,
            51.255,
        ],
        to_rgb=False)
'''
#as to_rgb=False, we swap the red and blue channel
input_array = input_array[:,[2, 1, 0], :, :]
#normalize image
mean = np.array([125.307, 122.961, 113.8575]).reshape(1, 3, 1, 1) 
std = np.array([51.5865, 50.847, 51.255]).reshape(1, 3, 1, 1) 
input_array = (input_array - mean) / std
input_array = input_array.astype('float32')

# Get the model's input name (usually 'input' or something similar)
input_name = sess.get_inputs()[0].name

# Run inference on the input image
outputs = sess.run(None, {input_name: input_array})

# Get the output (typically the class probabilities)
output = outputs[0]

# Print the output
print("Model output:", output)


Available providers: ['CPUExecutionProvider']
Model output: [[9.999151e-01 8.487269e-05]]


In [5]:
#Multi-label Classification 
from mmdeploy.apis import inference_model
result = inference_model(
    model_cfg='mmpretrain/work_dirs/resnet101-csra_1xb16_voc07-448px/resnet101-csra_1xb16_voc07-448px.py',
    deploy_cfg='mmdeploy/configs/mmpretrain/classification_onnxruntime_dynamic.py',
    backend_files=['mmdeploy_model/csra/end2end.onnx'],
    img='mmpretrain/data/VOC2007/JPEGImages/000001.jpg',
    device='cpu')
print(result)

[<DataSample(

META INFORMATION
    img_shape: (448, 448)
    ori_shape: (500, 353)
    num_classes: 20
    img_path: mmpretrain/data/VOC2007/JPEGImages/000001.jpg
    scale_factor: (1.2691218130311614, 0.896)

DATA FIELDS
    pred_score: tensor([1.4833e-04, 1.9765e-03, 4.1759e-04, 4.3038e-04, 1.6543e-03, 4.8158e-04,
        1.3101e-03, 6.9526e-04, 8.0482e-03, 1.7875e-04, 4.2096e-04, 9.9958e-01,
        1.2955e-04, 2.8720e-04, 9.9647e-01, 1.8178e-03, 9.2897e-04, 1.1288e-03,
        3.9685e-04, 2.9027e-03])
    pred_label: tensor([11, 14])

) at 0x7c70b43d5f60>]


In [6]:
#Multi-label Classification without torch (may have little difference with official)
import onnxruntime
import numpy as np
from PIL import Image
import cv2
'''
according to config
    dataset=dict(
        data_root='data/VOC2007',
        pipeline=[
            dict(type='LoadImageFromFile'),
            dict(scale=448, type='Resize'),
'''
# we resize
def resize(image, target_size):
    # Resize the image
    image = image.resize((target_size, target_size), Image.BILINEAR)

    return image
    
# Load the ONNX model with CUDA execution provider
sess = onnxruntime.InferenceSession("mmdeploy_model/csra/end2end.onnx", providers=["CPUExecutionProvider"])
print("Available providers:", sess.get_providers())

# Define the image path
image_path = "mmpretrain/data/VOC2007/JPEGImages/000001.jpg"

# Load and preprocess the image
image = Image.open(image_path)
image = resize(image, target_size=448)

# Convert the input tensor to a numpy array (ONNXRuntime uses numpy arrays)
input_array = np.transpose(image, (2,0,1))
input_array = np.expand_dims(input_array,axis=0)

'''
according to config
    data_preprocessor = dict(
        mean=[
            0,
            0,
            0,
        ],
        num_classes=20,
        std=[
            255,
            255,
            255,
        ],
        to_onehot=True,
        to_rgb=True)
'''
#normalize image
mean = np.array([0., 0., 0.]).reshape(1, 3, 1, 1) 
std = np.array([255., 255., 255.]).reshape(1, 3, 1, 1) 
input_array = (input_array - mean) / std
input_array = input_array.astype('float32')

# Get the model's input name (usually 'input' or something similar)
input_name = sess.get_inputs()[0].name

# Run inference on the input image
outputs = sess.run(None, {input_name: input_array})

# Get the output (typically the class probabilities)
output = outputs[0]

threshold = 0.5
# Print the output
print("Model output:", output)
print("Positive class:", np.where(output[0]>threshold)[0])

Available providers: ['CPUExecutionProvider']
Model output: [[1.5553832e-04 2.1567941e-03 4.2802095e-04 4.7284365e-04 1.6741157e-03
  5.2770972e-04 1.3337135e-03 7.0518255e-04 8.2379580e-03 1.9586086e-04
  4.7579408e-04 9.9962926e-01 1.3703108e-04 3.0830503e-04 9.9632323e-01
  2.0922124e-03 8.8834763e-04 1.1980832e-03 3.6549568e-04 3.4815073e-03]]
Positive class: [11 14]
